In [10]:
import pennylane as qml
from pennylane import numpy as np
import torch
from torch.autograd import Variable

In [68]:
num_qubits = 3

In [69]:
def build_hamiltonian(h):
    coeffs = []
    obs = []
    for i in range(num_qubits):
        for j in range(i+1, num_qubits):
            coeffs.append(-1)
            obs.append(qml.PauliZ(i) @ qml.PauliZ(j))
    for i in range(num_qubits):
        coeffs.append(-h)
        obs.append(qml.PauliX(i))
    hamiltonian = qml.Hamiltonian(coeffs, obs)
    return hamiltonian

In [84]:
dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev, interface="torch")
def circuit(params, hamiltonian):
    for i in range(num_qubits):
        qml.Rot(params[i * 3], params[i * 3 + 1], params[i * 3 + 2], wires=i)
    return qml.expval(hamiltonian)

@qml.qnode(dev)
def circuit_state(params):
    for i in range(num_qubits):
        qml.Rot(params[i * 3], params[i * 3 + 1], params[i * 3 + 2], wires=i)
    return qml.state()

In [85]:
test_params = torch.tensor([0, np.pi/2, 0] * num_qubits)

def cost(params, hamiltonian):
    return circuit(params, hamiltonian)

In [86]:
cost_check = cost(test_params, build_hamiltonian(0))
print(f"Energy: {cost_check}")

Energy: 0.0


In [87]:
def optimize(h=0):
    torch.manual_seed(56)
    params = Variable((np.pi * torch.rand(3 * num_qubits, dtype=torch.float64)), requires_grad=True)
    var_init = [params]
    hamiltonian = build_hamiltonian(h)
    cost_init = cost(params, hamiltonian)
    opt = torch.optim.SGD(var_init, lr=0.1)
    def closure():
        opt.zero_grad()
        loss = cost(params, hamiltonian)
        loss.backward()
        return loss

    var_pt = [var_init]
    cost_pt = [cost_init]
    x = [0]

    for i in range(100):
        opt.step(closure)
        if (i + 1) % 5 == 0:
            x.append(i)
            params_n = opt.param_groups[0]["params"][0]
            costn = cost(params_n, hamiltonian)
            var_pt.append([params_n])
            cost_pt.append(costn)

            # for clarity, the angles are printed as numpy arrays
            #print("Energy after step {:5d}: {: .7f} | Angles: {}".format(
            #    i+1, costn, params_n.detach().numpy()),"\n"
            #)
    state = circuit_state(params_n)
    print(f"FINAL ENERGY: {costn}, ANGLES: {params_n.detach().numpy()}\n")
    for i, s in enumerate(state.detach().numpy()):
        print(f"{i:0{num_qubits}b}: {abs(s) ** 2}")
    return state


In [88]:
state = optimize(0.2)
state = optimize(0.21)
state = optimize(0.22)
state = optimize(0.23)

FINAL ENERGY: -3.01372173594026, ANGLES: [1.96323939 3.21112746 2.34413844 0.65212549 3.22054849 2.48604503
 1.87199129 3.19192402 2.10144832]

000: 1.1917307950969227e-09
001: 1.8809504771859744e-06
010: 7.638678883638057e-07
011: 0.0012056394573642692
100: 9.851073079589788e-07
101: 0.001554829386475772
110: 0.0006314277035034398
111: 0.9966044723352521
FINAL ENERGY: -3.015226328292734, ANGLES: [1.96323939 3.2150007  2.34986893 0.65212549 3.22495371 2.49372249
 1.87199129 3.19397808 2.0966456 ]

000: 1.6036477123096257e-09
001: 2.3364113465399154e-06
010: 9.220178076756325e-07
011: 0.00134332051299636
100: 1.1893000210756135e-06
101: 0.0017327334690480484
110: 0.0006837884590758564
111: 0.9962357082260566
FINAL ENERGY: -3.0168300598446725, ANGLES: [1.96323939 3.21894398 2.35606258 0.65212549 3.22942443 2.50182611
 1.87199129 3.19599554 2.0919425 ]

000: 2.131571565683495e-09
001: 2.879398782306531e-06
010: 1.103819104756709e-06
011: 0.0014910760855003289
100: 1.4236102179294155e-06
1